In [98]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [99]:
# Load data and select trainging hour and test hour
trainhour = 6
testhour = 9
df = pd.read_csv('series.csv')
cols5 = ['21', '22', '23', '24', '25']
cols10 = ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
df['sum5'] = df[cols5].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [100]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x, y):
    if x < -3:
        if y < -10:
            return 1
        elif y > 10:
            return 2
        else:
            return 5
    elif x > 3:
        if y < 10:
            return 3
        elif y > 10:
            return 4
        else:
            return 5
    else:
        return 5

data["labels"] = data.apply(lambda x: setlabels(x.sum5, x.sum10), axis=1)
testdata["labels"] = testdata.apply(lambda x: setlabels(x.sum5, x.sum10), axis=1)

In [101]:
# Get X, y for model
X = data.iloc[:,8:28].to_numpy()

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = to_categorical(y_encoded)

In [102]:
# Split data into training and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [103]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               2100      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 5)                 505       
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


In [104]:
# Fit model
model.fit(X_train, y_train, epochs=5, shuffle=True, verbose=0)

In [105]:
# Model loss and Accuray
model_loss, model_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.5074039144679712, Accuracy: 0.837596595287323
Normal Neural Network - Loss: 0.5437814797726522, Accuracy: 0.8416580557823181


In [106]:
# Pull out validation data and make predicts
formodel = testdata.iloc[:,8:28].to_numpy()
testdata['predict'] = label_encoder.inverse_transform(model.predict_classes(formodel))
testdata = testdata.drop(testdata[(testdata['predict'] == 5)].index)

In [107]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    if ((testdata.iloc[i,totalcols-1] == 1) | (testdata.iloc[i,totalcols-1] == 2)):
        for j in range(10):
            temp -= testdata.iloc[i,28+j]
            if((temp < -2) | (temp > 15)):
              break
    else:
        for j in range(10):
            temp += testdata.iloc[i,28+j]
            if((temp < -2) | (temp > 15)):
              break
    profit.append(temp)

testdata['profit'] = profit
testdata['profit'].sum()

-19.29999999999999

In [108]:
testdata.to_csv('binarytest.csv')